<a href="https://colab.research.google.com/github/OluwatodimuAdegoke/ComputerVisionCourse/blob/main/Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import tensorflow as tf # Models
import pandas as pd  # Reading and processing data
import seaborn as sns # Visualization
from tensorflow.keras.layers import Normalization

Data Preparation

In [ ]:
# Read the data
data = pd.read_csv("train.csv",sep=',')
data.shape
# Plots the graphs of each variable against the other
sns.pairplot(data[['years','km','rating','condition','economy','top speed','hp', 'torque','current price']])

In [42]:
tensor_data = tf.constant(data)
# Cast to float32
tensor_data = tf.cast(tensor_data,tf.float32)
# Shuffle the data to avoid bias
tensor_data = tf.random.shuffle(tensor_data)


tf.Tensor(
[5.68000e+02 6.76727e+05 7.56074e+05 4.00000e+00 1.17511e+05 3.00000e+00
 1.00000e+00 1.30000e+01 1.39000e+02 1.20000e+02 1.23000e+02 2.46242e+05], shape=(12,), dtype=float32)


In [50]:
x = tensor_data[:,3:-1]
print(x.shape)
y = tensor_data[:,-1]
y = tf.expand_dims(y, axis = -1)
print(y.shape)

(1000, 8)
(1000, 1)


In [56]:
# Normalize the input
normalizer = Normalization()
# Gets the mean and std itself
normalizer.adapt(x)
normalizer(x)

<tf.Tensor: shape=(1000, 8), dtype=float32, numpy=
array([[-0.90849847, -0.7756557 ,  0.00855837, ..., -0.30567393,
        -0.07538996, -0.06760698],
       [-1.4904962 ,  1.3275927 , -0.7046627 , ...,  0.7317388 ,
         1.0461959 ,  0.69255364],
       [ 1.4194926 , -0.10140122,  0.00855837, ...,  0.42051497,
        -0.12415456,  0.50251347],
       ...,
       [-0.90849847, -0.8850054 , -1.4178838 , ...,  0.6279975 ,
         0.5585499 ,  1.5477343 ],
       [ 1.4194926 ,  0.43930662,  0.7217794 , ...,  0.05742051,
         1.6801357 , -1.6829482 ],
       [ 0.83749485, -0.737627  ,  0.00855837, ..., -0.0981914 ,
         0.21719766, -1.5879282 ]], dtype=float32)>